In [12]:
# TEAM MEMBERS

# Routhu Prashanth -CS19BTECH11042
# T Eshwar         -CS19BTECH11040
# Manjunath K      -EE19BTECH11015

# INPUT SPECIFICATIONS

# 1. Enter m(number of equations)
# 2. Enter n(number of variables)
# 3. Enter the matrix A as a list of entries one row followed by other each entry seperated by space
# 4. Enter vector b as space seperated entries
# 5. Enter vector c as space seperated entries

import numpy as np
from scipy.linalg import null_space as nl
def move_to_vertex(A,z,m,n,b,c):
  while 1:
    tight_indices=list(np.where(np.dot(A,z)==b)[0])
    untight_indices=list(np.where(np.dot(A,z)!=b)[0])
    tight_rows= A[tight_indices]
    tight_bs=b[tight_indices]
    untight_bs=b[untight_indices]
    untight_rows=A[untight_indices]
    if len(tight_rows)!=0 and np.linalg.matrix_rank(tight_rows)<n :
      rank=np.linalg.matrix_rank(tight_rows)
      if rank>0 and rank<n:
        direction=np.matrix.transpose(nl(tight_rows))[0]
        alpha=find_alpha(untight_rows,direction,z,untight_bs)
        z=z+alpha*direction
    elif len(tight_rows)!=0 and (np.linalg.matrix_rank(tight_rows)==n):
      break
    else:
      direction=c.copy()
      alpha=find_alpha(untight_rows,direction,z,untight_bs)
      z=z+alpha*direction

  while(is_degenerate(A,b,z)):
    b=perturb_input(tight_rows,b,tight_bs,tight_indices,z)
  return z,b

def vertexMarching(A,z,c,b,inp_b):
  while 1:
    tight_indices=list(np.where(np.dot(A,z)==b)[0])
    untight_indices=list(np.where(np.dot(A,z)!=b)[0])
    tight_rows= A[tight_indices]
    untight_rows=A[untight_indices]
    tight_bs=b[tight_indices]
    untight_bs=b[untight_indices]
    if(is_max(tight_rows,c)):
      break
    else:
      z=neighbour(tight_rows,untight_rows,c,z,untight_bs)
      if(len(z)==0):
        return None
      while is_degenerate(A,b,z):
        b=perturb_input(tight_rows,b,tight_bs,tight_indices,z)
  if(is_feasible(A,b,z)):
    print("The optimum vertex is:")
    print(z)
    print("Optimal value:")
    print(np.dot(c,z))
  else:
    z=initial_feasible_solution(A,inp_b,c)
    m=np.shape(A)[0]
    n=np.shape(A)[1]
    simplex(A,z,c,m,n,inp_b)



def neighbour(tight_rows,untight_rows,c,z,untight_bs):
  f_directions=-np.matrix.transpose(np.linalg.inv(tight_rows))
  v_index=np.where(np.dot(f_directions,c)>0)[0]
  direction=f_directions[v_index][0]
  l=[]
  for i in range(len(untight_rows)):
    if np.dot(direction,untight_rows[i])!=0:
      l.append(untight_bs[i]-1*(np.dot(z,untight_rows[i]))/(np.dot(direction,untight_rows[i])))
  l1=np.array(l)
  alpha_indices=np.where(l1>=0)
  positive_alphas=l1[alpha_indices]
  if(len(positive_alphas)==0):
    print("The given LP is unbounded")
    return []
  req_alpha=np.min(positive_alphas)
  return (z+req_alpha*direction)

def is_max(tight_rows,c):
  b1=-np.matrix.transpose(np.linalg.inv(tight_rows))
  p=np.dot(b1,c)
  positive_indices=np.where(p>0)[0]
  if(len(positive_indices)!=0):
    return False
  else:
    return True

def simplex(A,z,c,m,n,b):
  vertex,modified_b=move_to_vertex(A,z,m,n,b,c)
  vertexMarching(A,vertex,c,modified_b,b) 

def is_degenerate(A,b,z):
  n=np.shape(A)[1]
  tight_indices=list(np.where(np.dot(A,z)==b)[0])
  if(len(tight_indices)>n):
    return True
  else:
    return False

def is_feasible(A,b,x):
  eps=1e-4
  checker= np.dot(A,x)-b
  m=np.shape(A)[0]
  satisfied_constraints=np.where(checker<=eps)[0]
  if(len(satisfied_constraints)==m):
    return True
  else:
    return False

def initial_feasible_solution(A,b,c):
  indices=np.where(b>0)[0]
  m=np.shape(A)[0]
  n=np.shape(A)[1]
  if(len(indices)==len(b)):
    return np.zeros(np.shape(c))
  else:
    for i in range(m*100):
      rand_indices=np.random.choice(m,n,False)
      for_vertex=A[rand_indices]
      correspoding_bs=b[rand_indices]
      if(np.linalg.matrix_rank(for_vertex)==n):
        vertex=np.dot(np.linalg.inv(for_vertex),correspoding_bs)
        if(is_feasible(A,b,vertex)):
          return vertex
  print("The given LP is likely to be infeasible. Check again")
  k=[]
  return k

def perturb_input(tight_A,b,tight_bs,tight_indices,x):
  eps=1e-4
  A=tight_A
  m=np.shape(A)[0]
  n=np.shape(A)[1]
  rows_modified=m-n
  updated_b=b.copy()
  temp_b=tight_bs
  for i in range(m*100):
    for j in range(n,m):
      temp_b[i]=temp_b[i]+np.random.normal(eps,eps*10)
    i=0
    for k in tight_indices:
      updated_b[k]=temp_b[i]
      i=i+1
    return updated_b

def find_alpha(untight_rows,direction,z,untight_bs):
  l=[]
  for i in range(len(untight_rows)):
    if np.dot(direction,untight_rows[i])!=0:
      l.append(untight_bs[i]-1*(np.dot(z,untight_rows[i]))/(np.dot(direction,untight_rows[i])))
  l1=np.array(l)
  p=list(abs(l1))
  p1=p.index(np.min(p))
  return l1[0]

def main():
  m = int(input('Enter m(length of vector b) : '))
  n = int(input('Enter n(length of vector c) : '))
  A = np.array(list(map(float, input('Enter matrix A : ').split(' ')))).reshape(m, n)
  b = np.array(list(map(float, input('Enter vector b : ').split(' '))))
  c = np.array(list(map(float, input('Enter vector c : ').split(' '))))
  print(f" A = {A} \n b = {b} \n c = {c} \n ")
  z=initial_feasible_solution(A,b,c)
  simplex(A,z,c,m,n,b)

if __name__ == "__main__":
    main()

# A=np.array([[-1,-1],[-1,0],[0,-1],[0,1]])
# m=4
# n=2
# b=np.array([-4,-1.0,-1,3])
# c=np.array([-2,-3])
# z=initial_feasible_solution(A,b,c)
# # print(z)
# m=np.shape(A)[0]
# n=np.shape(A)[1]
# simplex(A,z,c,m,n,b)
# A=np.array([[1,1],
#             [-1,0],
#             [0,-1]])
# z=np.array([0.25,0])
# c=np.array([1,2])
# m=3
# n=2
# b=np.array([1,0,0])


Enter m(length of vector b) : 3
Enter n(length of vector c) : 2
Enter matrix A : 1 1 -1 0 0 -1
Enter vector b : 1 0 0
Enter vector c : 1 2
 A = [[ 1.  1.]
 [-1.  0.]
 [ 0. -1.]] 
 b = [1. 0. 0.] 
 c = [1. 2.] 
 
The optimum vertex is:
[0. 1.]
Optimal value:
2.0
